# Save land-only XCO2 data
---

In [1]:
import sys
sys.path.insert(0, "../src")

In [15]:
import numpy as np
import pandas as pd
import xarray
import regionmask

import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [3]:
DS_xco2 = xarray.open_dataset("../data/OCO2_Lite_XCO2.nc")
DS_xco2

<xarray.Dataset>
Dimensions:  (time: 182297008)
Coordinates:
    lon      (time) float32 ...
    lat      (time) float32 ...
  * time     (time) datetime64[ns] 2014-09-06T02:07:25.228014848 ... 2020-07-...
Data variables:
    xco2     (time) float32 ...

In [29]:
# Relabel lon-lat coords on 1-degree

# Establish grid
lon_bins = np.arange(-180, 180)
lat_bins = np.arange(-90, 90)
lon_centers = (lon_bins[1:] + lon_bins[:-1]) / 2
lat_centers = (lat_bins[1:] + lat_bins[:-1]) / 2

# Assign grid labels
df_grid_xco2 = DS_xco2.to_dataframe().reset_index()
df_grid_xco2["lon_centers"] = pd.cut(df_grid_xco2.lon, lon_bins, labels=lon_centers).astype(float)
df_grid_xco2["lat_centers"] = pd.cut(df_grid_xco2.lat, lat_bins, labels=lat_centers).astype(float)

df_grid_xco2

,time,xco2,lon,lat,lon_centers,lat_centers
0,2014-09-06 02:07:25.228014848,394.484650,160.511780,54.465744,160.5,54.5
1,2014-09-06 02:07:25.561990144,392.932068,160.501877,54.485321,160.5,54.5
2,2014-09-06 02:07:25.615017472,394.069733,160.537964,54.482136,160.5,54.5
3,2014-09-06 02:07:25.920990464,394.211243,160.510025,54.503334,160.5,54.5
4,2014-09-06 02:07:25.947986944,394.436035,160.528091,54.501705,160.5,54.5
...,...,...,...,...,...,...
182297003,2020-07-31 23:20:01.895003136,408.557220,-175.845139,70.306435,-175.5,70.5
182297004,2020-07-31 23:20:01.975026688,408.668915,-175.726501,70.288025,-175.5,70.5
182297005,2020-07-31 23:20:02.228012544,407.755005,-175.874786,70.321869,-175.5,70.5
182297006,2020-07-31 23:20:02.642011648,407.212646,-175.785629,70.318901,-175.5,70.5


In [24]:
# Construct ocean mask
land = regionmask.defined_regions.natural_earth.land_110
mask = land.mask(lon_centers, lat_centers)
# mask.plot(add_colorbar=False);

df_mask = mask.to_dataframe().reset_index().rename(columns={"lon": "lon_centers", "lat": "lat_centers"})
df_mask

,lat_centers,lon_centers,region
0,-89.5,-179.5,0.0
1,-89.5,-178.5,0.0
2,-89.5,-177.5,0.0
3,-89.5,-176.5,0.0
4,-89.5,-175.5,0.0
...,...,...,...
64256,88.5,174.5,NaN
64257,88.5,175.5,NaN
64258,88.5,176.5,NaN
64259,88.5,177.5,NaN


In [33]:
# Merge land mask, drop ocean values, convert to dataset

DS_temp = (
    pd.merge(df_grid_xco2, df_mask, on=["lon_centers", "lat_centers"], how="left")
    .dropna(subset=["region"])
    .drop(columns=["lon_centers", "lat_centers", "region"])
    .set_index(["time"])
    .to_xarray()
)

DS_xco2_land = xarray.Dataset(
    {"xco2": (["time"], DS_temp.xco2),},
    coords={
        "lon": (["time"], DS_temp.lon),
        "lat": (["time"], DS_temp.lat),
        "time": DS_temp.time,
    },
)

DS_xco2_land

<xarray.Dataset>
Dimensions:  (time: 58844887)
Coordinates:
    lon      (time) float32 160.51178 160.50188 ... -104.92578 -104.92716
    lat      (time) float32 54.465744 54.48532 54.482136 ... 54.38092 54.38522
  * time     (time) datetime64[ns] 2014-09-06T02:07:25.228014848 ... 2020-07-...
Data variables:
    xco2     (time) float32 394.48465 392.93207 394.06973 ... 407.81757 407.105

In [34]:
DS_xco2_land.to_netcdf("../data/OCO2_Lite_XCO2_land.nc", format="NETCDF4")